Django提高03
====
**调度和处理URL**

一个干净优雅的URL方案是高质量Web应用程序的重要细节之一。

Django允许你根据需要自由地设计URL，没有框架限制。

URLs应该是干净和可用的，不需要`.php`、`.cgi`或`0,2097,1-1-1928,00`这类荒谬的东西。

请参考来自于www的创始人Tim Berners-Lee的一篇文章：[Cool URIsdon’t change](https://www.w3.org/Provider/Style/URI)。作者就为什么URL应该是干净、优雅的，给出了充足和经典的理由。

# 概述

要为应用程序设计URL，可以创建一个Python模块，命名为URLconf（URL configuration, URL配置）。这个模块是纯Python代码，并且是URL路径表达式与Python函数（视图函数）之间的映射。

这种映射可以尽可能地满足需要的长短。它还可以引用其他映射。而且，由于它是纯粹的Python代码，因此可以动态地构建它。

Django还提供了根据活动语言翻译URL的方法。 有关更多信息，请参阅[internationalization documentation](https://docs.djangoproject.com/en/2.0/topics/i18n/translation/#url-internationalization)。

# Django怎样处理请求

当用户从Django支持的站点请求页面时，系统会根据如下规则和算法确定要执行的Python代码：

1.Django确定要使用的根URLconf模块。通常，这是ROOT_URLCONF设置的值，但是如果传入的HttpRequest对象具有urlconf属性（由中间件设置），则其值将用于替代ROOT_URLCONF设置。

>> 参考[HttpRequest通过app代码设置的属性](Django提高02-Request和Response对象.ipynb)HttpRequest.urlconf属性设置

2.Django 加载该 Python 模块，并查找变量`urlpatterns`。该变量是一个 Python 列表，`django.urls.path（）`和/或`django.urls.re_path（）`实例构成列表的元素。

3.Django按顺序遍历每个URL模式（urlpatterns列表中的每个元素），并停在与请求的URL匹配的第一个URL模式上。

4.一旦某个URL模式匹配，Django就会导入并调用给定的视图，该视图是一个简单的 Python 函数（或者是 class-based view）。视图函数获取以下参数：

> 一个HttpRequest实例。

>- 如果匹配的URL模式没有返回任何命名组，则来自正则表达式的匹配将作为位置参数提供。

>- Kwargs 参数由路径表达式匹配的命名部分组成，并由约定的一个参数重写，这个约定的参数是`django.urls.path（）`或`django.urls.re_path（）`中的可选参数。

>> **注：views的基类，有一个必需参数，一个位置参数，一个关键字参数。如def get(self, request, *args, **kwargs)**

5.如果没有URL模式匹配，或者在此过程中的任何点发生异常，Django将调用适当的错误处理视图。

**注意：HttpRequest由django在后台自动处理完成。将处理后的一个HttpRequest对象作为参数提供视图。**

# URLconf实例解析
下面是URLconf的一个示例：

```
from django.urls import path

from . import views

urlpatterns = [
    path('articles/2003/', views.special_case_2003),
    path('articles/<int:year>/', views.year_archive),
    path('articles/<int:year>/<int:month>/', views.month_archive),
    path('articles/<int:year>/<int:month>/<slug:slug>/', views.article_detail),
]
```

**注意：**

> 要从URL中捕获值，请使用尖括号`<...>`。

> 捕获的值可以包含一个类型转换器（可选项）。例如，使用`<int：name>`来捕获整数参数。 如果不包含转换器，捕获的值被匹配为不包括`/`符号的字符串。

>没有必要添加一个前导斜杠，因为每个URL都有这个斜杠。 例如，`articles`，而不是`/articles`。

**请求的示例：**

> 请求`/ articles / 2005/03 /`将与列表中的第三个条目匹配。 Django会调用函数views.month_archive（request，year = 2005，month = 3）。

> `/ articles / 2003 /`将匹配列表中的第一个模式，而不是第二个模式，因为这些模式是按顺序测试的，而第一个模式是第一个要传递的测试。 随意利用订单来插入像这样的特殊情况。 在这里，Django会调用函数`views.special_case_2003`（request）

> `/ articles / 2003`不符合任何这些模式，因为每个模式都要求URL以斜杠结尾。

> `/ articles / 2003/03 / building-a-django-site /`会匹配最后一个模式。 Django会调用函数views.article_detail（request，year = 2003，month = 3，slug =“building-a-django-site”）。

# 路径转换器

默认可以使用下列的路径转换器：
- str - 匹配任何非空字符串，不包括路径分隔符 '/'。 如果表达式中不包含转换器，这是默认值。
- int - 匹配零或任何正整数。 返回一个int。
- slug - 匹配任何由ASCII字母或数字组成的字符串，加上连字符和下划线字符。 例如，building-your-1st-django-site。
- uuid - 匹配格式化的UUID。 为防止多个URL映射到同一页面，必须包含破折号，并且字母必须是小写。 例如，075194d3-6885-417e-a8a8-6c931e272f00。 返回一个UUID实例。
- path- 匹配任何非空字符串，包括路径分隔符 '/'。 这可以匹配一个完整的URL路径，而不仅仅是一个URL路径的一部分，就像str一样。

# 自定义路径转换器
对于更复杂的匹配要求，您可以定义自己的路径转换器。

转换器是一个包含以下内容的类：
- 一个正则表达式类的属性，作为一个字符串。
- 一个`to_python（self，value）`方法，它将匹配的字符串转换为一个类型，这个类型将被传递给视图函数。 如果它不能转换给定的值，将引发`ValueError`。
- `to_url（self，value）`方法，它将Python类型转换为URL中使用的字符串。

**自定义转换器示例：**
```
class FourDigitYearConverter:
    regex = '[0-9]{4}'

    def to_python(self, value):
        return int(value)

    def to_url(self, value):
        return '%04d' % value
```

**注册自定义的转换器**

在URLconf中注册自定义转换器类，使用`register_converter()`方法:

```
from django.urls import register_converter, path

from . import converters, views

register_converter(converters.FourDigitYearConverter, 'yyyy')

urlpatterns = [
    path('articles/2003/', views.special_case_2003),
    path('articles/<yyyy:year>/', views.year_archive),
    ...
]
```

# 使用正则表达式

如果路径和转换器语法不足以定义您的URL模式，还可以使用正则表达式。 要使用正则表达式，请使用re_path（）而不是path（）。

在Python正则表达式中，命名正则表达式组的语法是`（？P <name> pattern）`，其中`name`是组的名称，`pattern`是一些要匹配的模式。

对前述示例`URLconf`，使用正则表达式重写如下：

```
from django.urls import path, re_path

from . import views

urlpatterns = [
    path('articles/2003/', views.special_case_2003),
    re_path(r'^articles/(?P<year>[0-9]{4})/$', views.year_archive),
    re_path(r'^articles/(?P<year>[0-9]{4})/(?P<month>[0-9]{2})/$', views.month_archive),
    re_path(r'^articles/(?P<year>[0-9]{4})/(?P<month>[0-9]{2})/(?P<slug>[\w-]+)/$', views.article_detail),
]
```

这完成了与前面的例子大致相同的事情，除了：

> 精确的URL，这种匹配会受到一些细微的限制。 例如，年份10000将不再匹配，因为年份整数限制为四位数字。

> 无论正则表达式匹配什么类型，每个捕获的参数都以字符串的形式发送到视图。

从使用`path（）`切换到`re_path（）`或反过来切换时，重点要注意视图参数的类型可能会更改，因此您可能需要调整视图。

# 使用未命名的正则表达式组
除了命名的组语法，例如 （？P <year> [0-9] {4}），您也可以使用较短的未命名组，例如（[0-9]{4}）。

不推荐这种用法，因为它可能更容易地意外地在匹配的意图和视图的参数之间引入错误。

无论哪种情况，建议在给定的正则表达式中只使用一种样式。 当两种样式混合使用时，所有未命名的组都会被忽略，只有命名组被传递给视图函数。

# 嵌套参数
正则表达式允许嵌套参数，Django将解析它们并将它们传递给视图。在反过来时，Django将尝试填充所有外部捕获的参数，而忽略任何嵌套的捕获参数。考虑下面的URL模式，它选择性地使用页面参数：

```
from django.urls import re_path

urlpatterns = [
    re_path(r'^blog/(page-(\d+)/)?$', blog_articles),                  # bad
    re_path(r'^comments/(?:page-(?P<page_number>\d+)/)?$', comments),  # good
]
```

这两种模式都使用了嵌套参数，将这样解析：例如，`blog / page-2 /`将生成与`blog_articles`的匹配，该匹配带有两个位置参数：`page-2 /`和`2`。

第二个模式用于评论，它将匹配`comments / page-2 /`的关键字参数`page_number`设置为2。在这种情况下，外部参数是非捕获参数（？：...）。

在这种情况下，`blog_articles`视图需要将最外面的捕获参数反转过来，`page-2/`和`2`，或没有参数，然而可以通过无参数或一个值为`page_number`反转`comments`。

嵌套的捕获参数在视图参数和URL之间创建了一个强耦合，例如`blog_articles`：视图接收部分URL（page-2 /），而不仅仅是视图中感兴趣的值。在反转时，耦合更加明显，因为要反转视图，我们需要传递一段URL而不是页码。

作为一个经验法则，只需捕获视图需要使用的值并在正则表达式需要参数但视图忽略它时使用非捕获参数。

# URLconf搜索的内容
URLconf作为普通的Python字符串来搜索请求的URL。这不包括GET或POST参数或域名。

例如：
> 在`https://www.example.com/myapp/`的请求中，URLconf将查找`myapp /`。

> 在`https://www.example.com/myapp/?page=3`的请求中，URLconf将查找`myapp /`。

URLconf不查看请求方法。 换句话说，所有的请求方法，`- POST，GET，HEAD` 等将被路由到同一个URL的同一个函数。

# 指定视图参数的默认值
一个方便的技巧是为视图的参数指定默认参数。下面是一个`URLconf` 和 `view` 的例子：

```
# URLconf
from django.urls import path

from . import views

urlpatterns = [
    path('blog/', views.page),
    path('blog/page<int:num>/', views.page),
]

# View (in blog/views.py)
def page(request, num=1):
    # Output the appropriate page of blog entries, according to num.
    ...
```

在上面的例子中，两个URL模式指向相同的`view - views.page` - 但是第一个模式并没有从URL中捕获任何东西。

如果第一个模式匹配，则`page（）`函数将使用`num`的缺省参数 `1`。如果第二个模式匹配，`page（）`将使用捕获的任何`num`值，例如'blog/page005'，将把'005'转换为整数 5 。

**性能**

第一次访问`urlpatterns`中的每个正则表达式都会被编译。 这使得系统非常快速。

**urlpatterns变量的语法**

`urlpatterns`应该是python的`path（）`和`/`或`re_path（）`的实例的列表。

# 错误处理
当Django无法为请求的URL找到匹配项或者引发异常时，Django会调用错误处理视图。

用于这种情况的视图通过四个变量指定。他们的默认值应该足以满足大多数项目，但是可以通过覆盖其默认值进一步定制。

有关详细信息，请参阅定制错误视图的文档。

这些值可以在您的根URLconf中设置。在其他URLconf中设置这些变量将不起作用。

值必须是可调用的，或者是字符串，该字符串代表完整的Python导入到视图的路径，该视图是当前被调用用于处理错误情况的视图。

四个变量是：

- handler400 - 请参阅django.conf.urls.handler400。
- handler403 - 请参阅django.conf.urls.handler403。
- handler404 - 请参阅django.conf.urls.handler404。
- handler500 - 请参阅django.conf.urls.handler500。

# 包括其他URLconfs
在任何时候，你的`urlpatterns`都可以“包含”其他URLconf模块。这实质上是将一组网址“植根于”其他网址之下。

例如，下面是Django网站本身的URLconf的摘录。 它包含许多其他URLconf

```
from django.urls import include, path

urlpatterns = [
    # ... snip ...
    path('community/', include('aggregator.urls')),
    path('contact/', include('contact.urls')),
    # ... snip ...
]
```

每当Django遇到`include（）`时，它会截断与该点匹配的URL的部分，并将剩余的字符串发送到included的URLconf以供进一步处理。

例如：在基础实例中，当用户在浏览器中访问：`.../tnt/pandas`时：

- 一级路由中`path('tnt/',include('tnt.urls')),` 匹配'tnt'；
- include()会将url从匹配位置截断，将剩余的'django'发送给includ()包含的'tnt.urls'，
- 在'tnt.urls'中查找与'pandas'匹配的项。

另一种可能性是通过使用`path（）`实例的列表来包含其他URL模式。 例如，考虑这个URLconf：

```
from django.urls import include, path

from apps.main import views as main_views
from credit import views as credit_views

extra_patterns = [
    path('reports/', credit_views.report),
    path('reports/<int:id>/', credit_views.report),
    path('charge/', credit_views.charge),
]

urlpatterns = [
    path('', main_views.homepage),
    path('help/', include('apps.help.urls')),
    path('credit/', include(extra_patterns)),
]
```

在这个例子中，`/ credit / reports / URL`将由`credit_views.report()` Django视图处理。

这可以用来消除重复使用单个模式前缀的URLconf中的冗余。 例如，考虑这个URLconf：

```
from django.urls import path
from . import views

urlpatterns = [
    path('<page_slug>-<page_id>/history/', views.history),
    path('<page_slug>-<page_id>/edit/', views.edit),
    path('<page_slug>-<page_id>/discuss/', views.discuss),
    path('<page_slug>-<page_id>/permissions/', views.permissions),
]
```

我们可以通过只声明一次公共路径前缀并对不同的后缀进行分组来改善这一点：

```
from django.urls import include, path
from . import views

urlpatterns = [
    path('<page_slug>-<page_id>/', include([
        path('history/', views.history),
        path('edit/', views.edit),
        path('discuss/', views.discuss),
        path('permissions/', views.permissions),
    ])),
]
```

# 捕获参数 Captured parameters
included的URLconf从父URLconf接收任何捕获的参数，因此以下示例是有效的：

```
# In settings/urls/main.py
from django.urls import include, path

urlpatterns = [
    path('<username>/blog/', include('foo.urls.blog')),
]

# In foo/urls/blog.py
from django.urls import path
from . import views

urlpatterns = [
    path('', views.blog.index),
    path('archive/', views.blog.archive),
]
```

在上面的例子中，如预期的那样，捕获的“username”变量被传递给包含的URLconf。

# 传递额外的选项给视图函数
Passing extra options to view functions

URLconf有一个钩子，允许你将额外的参数作为Python字典传递给你的视图函数。

`path（）`函数可以接受一个可选的第三个参数，它应该是一个额外的关键字参数字典传递给视图函数。

例如：

```
from django.urls import path
from . import views

urlpatterns = [
    path('blog/<int:year>/', views.year_archive, {'foo': 'bar'}),
]
```

在这个例子中，对于`/ blog / 2005 /`的请求，Django将调用`views.year_archive（request，year = 2005，foo ='bar'）`。

该技术用于联合框架以将元数据和选项传递给视图。

**处理冲突**

可能有一个URL模式捕获命名的关键字参数，并且还会在其额外参数字典中传递具有相同名称的参数。 发生这种情况时，将使用字典中的参数，而不是URL中捕获的参数。

# 传递额外选项给 include(）
同样，您可以将额外选项传递给include（），并且包含的URLconf中的每一行都将传递额外的选项。例如，这两个URLconf集在功能上是相同的：

**设置一个：**

```
# main.py
from django.urls import include, path

urlpatterns = [
    path('blog/', include('inner'), {'blog_id': 3}),
]

# inner.py
from django.urls import path
from mysite import views

urlpatterns = [
    path('archive/', views.archive),
    path('about/', views.about),
]
```

**设置两个**

```
# main.py
from django.urls import include, path
from mysite import views

urlpatterns = [
    path('blog/', include('inner')),
]

# inner.py
from django.urls import path

urlpatterns = [
    path('archive/', views.archive, {'blog_id': 3}),
    path('about/', views.about, {'blog_id': 3}),
]
```

请注意，无论该行的视图是否实际接受这些选项都是有效的，总是会将其他选项传递给包含的URLconf中的每一行。 出于这个原因，只有在确定包含的URLconf中的每个视图都接受了您传递的额外选项时，此技术才有用。

# 反向解析URL

处理Django项目时的一个常见需求是可以获取URL的最终形式，以嵌入生成的内容（视图和有用的URL，显示给用户的URL等），或者用于处理服务器端的导航流（重定向等）

强烈希望避免对这些URL进行硬编码（一种费力，不可扩展且容易出错的策略）。同样危险的是设计专门的机制来生成URL，而这个设计与URLconf描述的设计是并行L，这可能会导致生成随着时间的推移而变旧的URL。

换句话说，需要的是一个DRY机制。除了其他优势之外，它还允许进行URL设计的演变，而无需查看所有项目源代码来搜索和替换过时的URL。

**DRY（Don't repeat yourself），字面意思来看："不要重复自己"。强调的意思就是在进行编程时相同的代码不要重复写，最好只写一次，然后可以在其他地方直接引用。如此一来，可以提高代码重用率，缩减代码量，同时也有助于提高代码的可读性和可维护性。当需要做出更改时，只需要更改一个地方即可。**

我们可用于获取URL的主要信息是负责处理它的视图的标识（例如名称）。其他参与查找正确URL的必需信息有视图参数的类型（位置，关键字）和值。

Django提供了一个解决方案，使得URL映射器是URL设计的唯一存储库。你用你的URLconf提供它，然后它可以在两个方向上使用：

> 从用户/浏览器请求的URL开始，它调用正确的Django视图，提供它可能需要的任何参数以及从URL中提取的值。

> 从相应的Django视图的标识以及将传递给它的参数的值开始，获取关联的URL。

第一个是我们在前几节讨论过的用法。第二种是所谓的URL反向解析，反向URL匹配，反向URL查询或者仅仅URL反转。

Django提供了用于执行URL反转的工具，以匹配需要URL的不同层：

- 在模板中：使用网址模板标签。
- 在Python代码中：使用reverse（）函数。
- 在与处理Django模型实例的URL相关的更高级代码中：get_absolute_url（）方法。

**反向解析示例**

再考虑一下URLCONF条目：

```
from django.urls import path

from . import views

urlpatterns = [
    #...
    path('articles/<int:year>/', views.year_archive, name='news-year-archive'),
    #...
]
```

根据此设计，对应于年nnnn的档案的URL是`/ articles / <nnnn> /`。

您可以使用以下方式在模板代码中获得这些内容:

```
<a href="{% url 'news-year-archive' 2012 %}">2012 Archive</a>
{# Or with the year in a template context variable: #}
<ul>
{% for yearvar in year_list %}
<li><a href="{% url 'news-year-archive' yearvar %}">{{ yearvar }} Archive</a></li>
{% endfor %}
</ul>
```

或者是如下的Python代码：

```
from django.urls import reverse
from django.http import HttpResponseRedirect

def redirect_to_year(request):
    # ...
    year = 2006
    # ...
    return HttpResponseRedirect(reverse('news-year-archive', args=(year,)))
```

如果出于某种原因决定应该更改每年发布文章内容的URL，那么您只需要更改URLconf中的条目。

在某些情况下，视图具有通用性，因此URL和视图之间可能存在多对一的关系。 对于这些情况，视图名称在倒转URL时不是一个足够好的标识符。 阅读下一节以了解Django为此提供的解决方案。

# 命名URL模式 Naming pattermns
为了执行URL反转，您需要使用上述示例中所做的命名的URL模式。用于URL名称的字符串可以包含您喜欢的任何字符。您不限于有效的Python名称。

命名URL模式时，请选择不太可能与其他应用程序的名称选择冲突的名称。如果您调用URL模式注释并且其他应用程序执行相同操作，则`reverse（）`查找的URL取决于项目的`urlpatterns`列表中最后一个模式。

在您的URL名称上加上一个来自应用程序名的前缀（例如myapp-comment，而不是comment）会降低冲突的可能性。

如果您想覆盖视图，您可以故意选择与另一个应用程序相同的URL名称。例如，一个常见的用例是重写LoginView。部分Django和大多数第三方应用程序采取具有这个名称的URL登录模式。如果你有一个自定义的登录视图，并给出它的URL登录名称，`reverse（）`会找到你的自定义视图，只要它被`included`在`django.contrib.auth.urls`后面的`urlpatterns`里（如果它被完全included）。

如果参数不同，您也可以为多个URL模式使用相同的名称。除了URL名称，`reverse（）`匹配参数的数量和关键字参数的名称。

# URL名称空间
## 介绍
即使不同的应用程序使用相同的URL名称，URL名称空间也允许您唯一地反转命名的URL模式。对于第三方应用程序来说，总是使用名称空间的URL是一种很好的做法（正如我们在本教程中所做的那样）。同样，它也允许您在部署应用程序的多个实例时反转URL。换句话说，由于单个应用程序的多个实例将共享命名的URL，因此命名空间提供了一种方法来区分这些命名的URL。

正确使用URL命名空间的Django应用程序可以为特定站点多次部署。例如`django.contrib.admin`有一个`AdminSite`类，它允许您轻松部署管理员的多个实例。在后面的示例中，我们将讨论在两个不同位置部署投票应用程序的想法，以便我们可以为两个不同的受众（作者和发布者）提供相同的功能。

一个URL命名空间有两部分，它们都是字符串：

### 应用程序名称空间
这描述了正在部署的应用程序的名称。每个应用程序的每个实例都具有相同的应用程序名称空间。例如，Django的管理应用程序具有可预测的应用程序名称空间`“admin”`。
### 实例名称空间
这标识了应用程序的特定实例。实例名称空间在整个项目中应该是唯一的。但是，实例名称空间可以与应用程序名称空间相同。这用于指定应用程序的默认实例。例如，默认的Django管理实例有一个名为'admin'的实例名称空间。
命名空间URL使用'：'运算符指定。例如，管理应用程序的主索引页面使用'admin：index'引用。这表示'admin'的命名空间和'index'的命名URL。

名称空间也可以嵌套。命名的URL`'sports：polls：index'`将在名为`'polls'`的命名空间中查找名为`'index'`的模式，该模式本身在顶级命名空间`'sports'`中定义。

## 反转命名空间URL
当给定名称空间URL（例如'polls：index'）来解析时，Django将整个限定名称拆分为多个部分，然后尝试以下查找：

1. 首先，Django查找匹配的应用程序名称空间（在本例中为'polls'）。这将产生该应用程序的实例列表。

2. 如果定义了当前的应用程序，Django将查找并返回该实例的URL解析器。当前应用程序可以使用reverse（）函数的current_app参数指定。

url模板标签使用当前解析视图的名称空间作为RequestContext中的当前应用程序。您可以通过在request.current_app属性上设置当前应用程序来覆盖此默认值。

3. 如果没有当前的应用程序。 Django寻找一个默认的应用程序实例。默认应用程序实例是具有与应用程序名称空间相匹配的实例名称空间的实例（在本例中，名为“polls”的polls实例）。

4. 如果没有默认的应用程序实例，Django会选择应用程序的最后一个部署实例，无论它的实例名称是什么。

5. 如果提供的名称空间在步骤1中与应用程序名称空间不匹配，则Django将尝试直接查找名称空间作为实例名称空间。

如果有嵌套命名空间，则对命名空间的每个部分重复这些步骤，直到只有视图名称未解析。视图名称将被解析为已找到的名称空间中的URL。

**命名空间示例**

为了展示这个解决方案的实际应用，请考虑本教程中两个投票应用程序实例的例子：一个名为“author-polls”，一个名为“publisher-polls”。 假设我们已经增强了该应用程序，以便在创建和显示民意测验时考虑实例命名空间。

```
# urls.py
from django.urls import include, path

urlpatterns = [
    path('author-polls/', include('polls.urls', namespace='author-polls')),
    path('publisher-polls/', include('polls.urls', namespace='publisher-polls')),
]
```

```
# polls/urls.py
from django.urls import path

from . import views

app_name = 'polls'
urlpatterns = [
    path('', views.IndexView.as_view(), name='index'),
    path('<int:pk>/', views.DetailView.as_view(), name='detail'),
    ...
]
```

使用此设置，可以进行以下查找： 

如果其中一个是当前实例 - 也就是说，如果我们在实例'author-polls'中呈现详细信息页面 - 'poll：index'将解析为'author-polls'实例的索引页面; 即后面的两项都会在“/ author-polls /”中生成。 

在基于类的视图的方法中：

```
reverse('polls:index', current_app=self.request.resolver_match.namespace)
```

并且在模板中：
`{% url 'polls:index' %}`

如果没有当前实例 - 也就是说，如果我们在网站的其他位置渲染页面 - “polls:index”将解析为最后一次注册的polls实例。 由于没有默认实例（“polls”实例的名称空间），因此将使用已注册的polls的最后一个实例。 这将是'publisher-polls'，因为它是在urlpatterns中声明的最后一个。

'author-polls：index'将始终解析为实例'author-polls'的索引页面（同样也适用于'publisher-polls'）。

如果还有一个默认实例 - 即一个名为“polls”的实例 - 则只有在没有当前实例（上面列表中的第二个项目）的情况下，上面的唯一更改才会发生。 在这种情况下，'poll：index'将解析为默认实例的索引页，而不是url模式中最后声明的实例。

## URL名称空间和包含的URLconf
包含的URLconf的应用程序名称空间可以用两种方式指定。

首先，您可以在包含的URLconf模块中设置app_name属性，与urlpatterns属性的级别相同。 您必须将实际的模块或模块的字符串引用传递给include（），而不是urlpatterns本身的列表。

**polls/urls.py**
```
from django.urls import path

from . import views

app_name = 'polls'
urlpatterns = [
    path('', views.IndexView.as_view(), name='index'),
    path('<int:pk>/', views.DetailView.as_view(), name='detail'),
    ...
]
```

**urls.py**
```
from django.urls import include, path

urlpatterns = [
    path('polls/', include('polls.urls')),
]
```

`polls.urls`中定义的URL将具有polls应用程序名称空间。

其次，您可以 include 一个包含嵌入名称空间数据的对象。 如果 `include(）`一个 path（）或 re_path（）实例的列表，则包含在该对象中的URL将被添加到全局名称空间。 但是，您也可以include（）一个2元元组，其中包含：

`(<list of path()/re_path() instances>, <application namespace>)`

**示例**
```
from django.urls import include, path

from . import views

polls_patterns = ([
    path('', views.IndexView.as_view(), name='index'),
    path('<int:pk>/', views.DetailView.as_view(), name='detail'),
], 'polls')

urlpatterns = [
    path('polls/', include(polls_patterns)),
]
```

这将包括提名的URL模式到指定的应用程序名称空间中。

实例名称空间可以使用`include（）`的名称空间参数来指定。 如果未指定实例名称空间，它将默认为包含的URLconf的应用程序名称空间。 这意味着它也将是该名称空间的默认实例。